<a href="https://colab.research.google.com/github/Abhisheksingh12345/ADT/blob/master/dev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plts

In [54]:
spark = SparkSession.builder.appName('dev_1').master('local[4]').getOrCreate()

In [ ]:
sc = spark.sparkContext

In [ ]:
table_A_path = '/content/drive/MyDrive/datasets/Table_A.csv'
table_B_path = '/content/drive/MyDrive/datasets/Table_B.csv'
table_C_path = '/content/drive/MyDrive/datasets/Table_C.csv'

In [ ]:
table_A_df = spark.read.format('csv').option('inferSchema',True).option('header',True).load(table_A_path)

In [ ]:
table_B_df = spark.read.format('csv').option('inferSchema',True).option('header',True).load(table_B_path)

In [22]:
table_C_df = spark.read.format('csv').option('inferSchema',True).option('header',True).load(table_C_path)

In [ ]:
table_A_df.coalesce

<bound method DataFrame.coalesce of DataFrame[cif_id: double,  account_no: string,  amount: double,  department: string,   Pride_rating: double, _c5: string]>

In [50]:
withoutscpace_tableC_df = table_C_df.na.fill("").withColumn("open_date",date_format(to_date(col("open_date"), "dd-MM-yyyy"), "yyyy-MM-dd"))

In [ ]:
withoutscpace_tableC_df.show()

In [ ]:
#@title Q3 Merge table A table B if A, loan no is null and last_date payroll >= 2024 then delete whole row
Q_3_df = table_A_df.join(table_B_df, on=["cif_id"], how="left_outer") \
  .filter((table_B_df["loan_no"].isNull()) & (table_B_df["last_date_for_payment"] >= "2024-01-01"))
Q_3_df.show()
Q_3_= table_A_df.join(Q_3_df.select("cif_id"), on=["cif_id"], how="left_anti")
Q_3.show()


In [ ]:
#@title Q6 For table C create a new Column which have open_date in format YYYMMM
Q6_df = withoutscpace_tableC_df.withColumn("new date", date_format(col('open_date'), 'yyyyMM'))
Q6_df.show()

In [ ]:
#@title Q7 For table A group by deparment order of all column amonut > 1L
Q_7 = table_A_df.filter(col('amount') > 100000).groupBy(col('department')).agg(count(col('cif_id')).alias('count')).orderBy('count')
Q_7.show()